In [14]:
import os,sys

# Get dir name of parent directory
modfol=os.path.dirname(os.getcwd())
sys.path.insert(0, modfol)

# Complete import statements 
import GeneralFunctions as GF, numpy as np, matplotlib.pyplot as plt, pandas as pd

# Set Data directory
datadir=modfol+"/Data/"

# Setname of observations file
obsfile=datadir+"Pyramid_SCol.xlsx" # Sheetnames = "Pyramid_hour"; "SCol"


# Comparison of Daily Observations and Reanalysis Data

First we import the weather station data. These data have a temporal frequency of 10 minutes, and are in Nepali time (GMT+5.45). We import into a pandas DataFrame and set the Date (first column) as the time index

In [22]:
obs=pd.read_excel(obsfile,sheetname="SCol",index_col=0,parse_dates=True,names=["Temp","Press"])
obs.head(5)

,Temp,Press
Date,,
2008-05-15 15:20:00,-16.8,382.5
2008-05-15 15:30:00,-17.4,382.4
2008-05-15 15:40:00,-17.8,382.5
2008-05-15 15:50:00,-17.7,382.2
2008-05-15 16:00:00,-17.2,382.1
